In [1]:
%pip install PyPDF2

In [2]:
import os
import PyPDF2
import re
from pathlib import Path
import xml.etree.ElementTree as ET
import re
import os

In [4]:
!git clone https://github.com/kermitt2/grobid_client_python


Cloning into 'grobid_client_python'...
remote: Enumerating objects: 347, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 347 (delta 113), reused 102 (delta 91), pack-reused 183
Receiving objects: 100% (347/347), 1.49 MiB | 8.82 MiB/s, done.
Resolving deltas: 100% (204/204), done.


In [5]:
%cd grobid_client_python


/Users/rutu/Downloads/NEU/SEM3/BigData/Assignment2/Demo/grobid_client_python


In [6]:
!python3 setup.py install

running install
/Users/rutu/opt/anaconda3/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/Users/rutu/opt/anaconda3/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
creating grobid_client_python.egg-info
writing grobid_client_python.egg-info/PKG-INFO
writing dependency_links to grobid_client_python.egg-info/dependency_links.txt
writing entry points to grobid_client_python.egg-info/entry_points.txt
writing requirements to grobid_client_python.egg-info/requires.txt
writing top-level names to grobid_client_python.egg-info/top_level.txt
writing manifest file 'grobid_client_python.egg-info/SOURCES.txt'
reading manifest file 'grobid_client_python.egg-info/SOURCES.t

In [8]:
!docker run -t --rm -p 8070:8070 lfoppiano/grobid:0.8.0

WARN  [2024-02-14 03:10:15,505] org.hibernate.validator.internal.properties.javabean.JavaBeanExecutable: HV000254: Missing parameter metadata for ResponseMeteredLevel(String, int), which declares implicit or synthetic parameters. Automatic resolution of generic type information for method parameters may yield incorrect results if multiple parameters have the same erasure. To solve this, compile your code with the '-parameters' flag.
[Wapiti] Loading model: "/opt/grobid/grobid-home/models/affiliation-address/model.wapiti"
Model path: /opt/grobid/grobid-home/models/affiliation-address/model.wapiti
[Wapiti] Loading model: "/opt/grobid/grobid-home/models/name/header/model.wapiti"
Model path: /opt/grobid/grobid-home/models/name/header/model.wapiti
[Wapiti] Loading model: "/opt/grobid/grobid-home/models/name/citation/model.wapiti"
Model path: /opt/grobid/grobid-home/models/name/citation/model.wapiti
[Wapiti] Loading model: "/opt/grobid/grobid-home/models/header/model.wapiti"
Model path: /opt

In [35]:
## Extract text using PyPDF2

import os
import PyPDF2

def extract_text_pypdf(pdf_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
    output_filename = os.path.join(output_folder, f'PyPDF_RR_{os.path.basename(pdf_path)[:-4]}_combined.txt')
    with open(output_filename, 'w') as file:
        file.write(text)
    print(f"Text extracted from {pdf_path} using PyPDF2 and saved to {output_filename}")


In [36]:
## Use Grobid to convert PDF to XML

import subprocess

def convert_to_xml_with_grobid(input_folder, output_folder):
    subprocess.run(['python3', '-m', 'grobid_client.grobid_client', '--input', input_folder, '--output', output_folder, 'processFulltextDocument'])
    print(f"PDF files in {input_folder} converted to XML and saved to {output_folder}")


In [37]:
## Convert XML to text using Grobid

def convert_xml_to_text(input_folder, output_folder):
    for xml_file in os.listdir(input_folder):
        if xml_file.endswith('.xml'):
            subprocess.run(['python3', '-m', 'grobid_client.grobid_client', '--input', os.path.join(input_folder, xml_file), '--output', os.path.join(output_folder, f'Grobid_RR_{xml_file[:-4]}_combined.txt'), 'processFulltextDocument'])
    print(f"XML files in {input_folder} converted to text and saved to {output_folder}")


In [38]:
## Run the extraction process

pdf_folder = '/Users/rutu/Downloads/NEU/SEM3/BigData/Assignment2/Archive 2'
output_folder_pypdf = '/Users/rutu/Downloads/NEU/SEM3/PyPDF'
output_folder_grobid = '/Users/rutu/Downloads/NEU/SEM3/GROBID'

for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        extract_text_pypdf(os.path.join(pdf_folder, filename), output_folder_pypdf)



Text extracted from /Users/rutu/Downloads/NEU/SEM3/BigData/Assignment2/Archive 2/2024-l3-topics-combined-2.pdf using PyPDF2 and saved to /Users/rutu/Downloads/NEU/SEM3/PyPDF/PyPDF_RR_2024-l3-topics-combined-2_combined.txt
Text extracted from /Users/rutu/Downloads/NEU/SEM3/BigData/Assignment2/Archive 2/2024-l1-topics-combined-2.pdf using PyPDF2 and saved to /Users/rutu/Downloads/NEU/SEM3/PyPDF/PyPDF_RR_2024-l1-topics-combined-2_combined.txt
Text extracted from /Users/rutu/Downloads/NEU/SEM3/BigData/Assignment2/Archive 2/2024-l2-topics-combined-2.pdf using PyPDF2 and saved to /Users/rutu/Downloads/NEU/SEM3/PyPDF/PyPDF_RR_2024-l2-topics-combined-2_combined.txt
output directory does not exist but will be created: /Users/rutu/Downloads/NEU/SEM3/GROBID
Successfully created the directory /Users/rutu/Downloads/NEU/SEM3/GROBID
GROBID server does not appear up and running, the connection to the server failed
PDF files in /Users/rutu/Downloads/NEU/SEM3/BigData/Assignment2/Archive 2 converted to X

In [43]:
convert_to_xml_with_grobid(pdf_folder, output_folder_grobid)
convert_xml_to_text(output_folder_grobid, output_folder_grobid)


GROBID server is up and running
Processing of /Users/rutu/Downloads/NEU/SEM3/BigData/Assignment2/Archive 2/2024-l2-topics-combined-2.pdf failed with error 408 , None
Processing of /Users/rutu/Downloads/NEU/SEM3/BigData/Assignment2/Archive 2/2024-l1-topics-combined-2.pdf failed with error 408 , None
Processing of /Users/rutu/Downloads/NEU/SEM3/BigData/Assignment2/Archive 2/2024-l3-topics-combined-2.pdf failed with error 408 , None
runtime: 60.015 seconds 
PDF files in /Users/rutu/Downloads/NEU/SEM3/BigData/Assignment2/Archive 2 converted to XML and saved to /Users/rutu/Downloads/NEU/SEM3/GROBID
XML files in /Users/rutu/Downloads/NEU/SEM3/GROBID converted to text and saved to /Users/rutu/Downloads/NEU/SEM3/GROBID
